In [ ]:
def get_latest_posts(limits):
    reddit = praw.Reddit(
        client_id="gK4gpkYtFvY1UMyGphPBzw",
        client_secret="KyeJsjXJpyYQZpLgbAXMuWQ8Wlgr3w",
        user_agent="reddit-stream"
    )
    subreddit = reddit.subreddit("MachineLearning")

    posts = []
    for post in subreddit.top(limit=limits):
        posts.append({
            "username": post.author.name if post.author else "deleted",
            "title": post.title,
            "upvotes": post.ups if hasattr(post, "ups") else 0,
            "downvotes": post.downs if hasattr(post, "downs") else "0",
            "num_comments": post.num_comments,
            "created_utc": datetime.datetime.fromtimestamp(post.created_utc),
        })

    df = pd.DataFrame(posts)
    df["created_utc"] = df["created_utc"].astype(str)
    return df

In [ ]:
redditdata = get_latest_posts(1000)

In [ ]:
redditdata

In [ ]:
redditdata.to_csv("reddit_data.csv", index=False)

In [22]:
import praw
import datetime
import pandas as pd
import re

In [23]:
df = pd.read_csv("reddit_data.csv")
data = df[['username', 'title', 'upvotes']].copy()
data.head()

,username,title,upvotes
0,cyrildiagne,[Project] From books to presentations in 10s w...,8437
1,TheInsaneApp,[D] A Demo from 1993 of 32-year-old Yann LeCun...,6283
2,programmerChilli,[R] First Order Motion Model applied to animat...,4886
3,TheInsaneApp,[D] This AI reveals how much time politicians ...,4876
4,TheInsaneApp,[N] AI can turn old photos into moving Images ...,4799


In [24]:
#tag = ['[D]','[R]','[N]','[P]','[d]','[r]','[n]','[p]','[Discussion]','[Project]','[News]','[Research]']
#pattern = '|'.join(re.escape(topic) for  in tag)
#data['topic'] = data['title'].str.findall(pattern)
#data['title'] = data['title'].str.replace(pattern, '',regex=True).str.strip()
#data['topic'] = data['topic'].apply(lambda x: ', '.join(x) if x else 'None')

In [25]:
tag_map = {
    '[D]': 'D', '[d]': 'D', '[Discussion]': 'D',
    '[R]': 'R', '[r]': 'R', '[Research]': 'R',
    '[N]': 'N', '[n]': 'N', '[News]': 'N',
    '[P]': 'P', '[p]': 'P', '[Project]': 'P',
}

pattern = '|'.join(re.escape(tags) for tags in tag_map)
#extract tag into another column
data['tag'] = data['title'].str.findall(pattern)
def normalize_tag(tag_list):
    if not tag_list:
        return 'None'
    normalized = list(set(tag_map[tags] for tags in tag_list))
    return ', '.join(sorted(normalized))

data['tag'] = data['tag'].apply(normalize_tag)
data['title'] = data['title'].str.replace(pattern, '',regex=True).str.strip()

In [26]:
data

,username,title,upvotes,tag
0,cyrildiagne,From books to presentations in 10s with AR + ML,8437,P
1,TheInsaneApp,A Demo from 1993 of 32-year-old Yann LeCun sho...,6283,D
2,programmerChilli,First Order Motion Model applied to animate pa...,4886,R
3,TheInsaneApp,This AI reveals how much time politicians star...,4876,D
4,TheInsaneApp,AI can turn old photos into moving Images / Li...,4799,N
...,...,...,...,...
992,sergeyfeldman,Which Machine Learning Classifiers are best fo...,353,P
993,milaworld,Alan Turing's “Intelligent Machinery” (1948),357,D
994,krallistic,Ali Rahimi's talk at NIPS(NIPS 2017 Test-of-ti...,355,N
995,deleted,Hopfield Networks is All You Need,353,R


In [21]:
data.to_csv("check1.csv", index=False)